In [2]:
# Author : Ghjulia Sialelli
# Date : 10.03.2022
# Summary : applying the reference projected pop. data disaggregated by broad age groups and sex to an example file.


# You will need to download the following files : 
# - Population_Age_Sex_2020.xlsx
# - End of 2020 conflict stock by age.xlsx

# and execute this notebook in the same folder.

In [3]:
# Necessary imports

import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None

# Global variable(s)

age_groups = ['0-4','0-17','5-14','15-17','15-24','25-64','65+','0+'] # age groups we are interested in for the GRID

## Example: applying the projection reference data to 2020 Conflict data.

### Loading reference data

We first load the reference data, downloadable [here](https://github.com/idmc-labs/ghjulia-internship/blob/main/PopDataDisaggregation/Population_Age_Sex_2020.xlsx).

In [4]:
final_df = pd.read_excel('Population_Age_Sex_2020.xlsx')

In [5]:
display(final_df)

,iso3,idmc_short_name,GRID_geographical_group,sex,0-4,0-17,5-14,15-17,15-24,25-64,65+,0+
0,AFG,Afghanistan,South Asia,Female,7.103221,23.969642,13.312439,3.553982,10.689513,16.164003,1.415506,48.684684
1,AFG,Afghanistan,South Asia,Male,7.468016,25.155873,13.941838,3.746018,11.290654,17.386837,1.227971,51.315316
2,AFG,Afghanistan,South Asia,Total,14.571238,49.125515,27.254277,7.300000,21.980167,33.550840,2.643477,100.000000
3,ALB,Albania,Europe and Central Asia,Female,2.778743,10.027579,5.383814,1.865023,7.085794,26.154915,7.676276,49.079542
4,ALB,Albania,Europe and Central Asia,Male,3.021883,11.239041,6.078499,2.138659,7.745745,27.023272,7.051059,50.920458
...,...,...,...,...,...,...,...,...,...,...,...,...
595,ZMB,Zambia,Sub-Saharan Africa,Female,7.925801,25.392760,13.882391,3.584567,10.547789,16.830768,1.300114,50.486863
596,ZMB,Zambia,Sub-Saharan Africa,Male,8.099875,25.775684,14.110863,3.564946,10.493391,15.982157,0.826851,49.513137
597,ZWE,Zimbabwe,Sub-Saharan Africa,Male,7.096731,24.477600,13.944572,3.436298,10.016144,15.585901,1.083008,47.726355
598,ZWE,Zimbabwe,Sub-Saharan Africa,Female,7.016010,24.264193,13.850397,3.397787,10.278488,19.198170,1.930580,52.273645


###### Description: Percentage of the population by broad age groups and sex (per 100 total population).

### Loading and pre-processing the example data

In this example, we focus on 2020 Conflict stock data, downloadable [here](https://github.com/idmc-labs/ghjulia-internship/blob/main/PopDataDisaggregation/End%20of%202020%20conflict%20stock%20by%20age.xlsx).

Please note that the following code is based on this specific file. Other examples will need a different tailoring of the code. To apply this code to another file, you will also need to change the following variables: `example_filename`, `sheet_name`, `skiprows`, `countries_col`, and `stock_col`. 

In [6]:
# name of the example file
example_filename = 'End of 2020 conflict stock by age.xlsx' 
stock_df = pd.read_excel(example_filename, sheet_name='IDP Data')

# name of the column in the .xlsx file that contains the countries isocodes
countries_col = 'iso3' 

# name of the column in the .xlsx file that contains the stock data
stock_col = 'All IDPs end 2020' 

In [7]:
display(stock_df[['iso3', countries_col, stock_col]].head())

,iso3,iso3,All IDPs end 2020
0,AFG,AFG,3546858
1,AZE,AZE,735455
2,BGD,BGD,426763
3,BEN,BEN,3494
4,BIH,BIH,98574


We check whether have a one-to-one mapping between the stock file and the reference file.


In [8]:
stock_countries = np.unique([str(country).strip() for country in stock_df[countries_col].values])
ref_countries = np.unique(final_df['iso3'].values)
diff = np.setdiff1d(stock_countries, ref_countries)

if len(diff) != 0 : print("\x1b[31m\"CAUTION : There are countries in the stock file that are not present in the reference file.\"\x1b[0m")

While it is not the case in this example, you will need to deal manually with those edge cases.

For example, you can remove some countries with the command 

``stock_df = stock_df[stock_df[countries_col].str.contains('MEX') == False]``

or rename them with 

``stock_df.loc[stock_df[countries_col] == 'MX', countries_col] = 'MEX'``

### Performing the projection

In [9]:
# Countries in the stock data 
countries = stock_df[countries_col].values

# Stock values
stocks = stock_df[stock_col].values

# Mapping from country (idmc_short_name) to stock
mapping = dict(zip(countries, stocks))

In [10]:
def rounding(x):
	"""

	Rounding function.

	The following are the rules that define all rounding carried out for IDMC's publications:
	- If the number is less than 100: report the number itself (i.e. 17 people)
	- If the number is between 100 and 999 - round to nearest 10 (i.e. 240 people)
	- If the number is between 1,000 and 9,999 - round to nearest 100 (i.e. 2,300 people )
	- If the number is over 10,000 - round to nearest 1,000 (i.e. 347,000 people)
	- If we want to report a ﬁgure rounding to the nearest million (mainly for aggregate ﬁgures) - round to the nearest 100,000 so that we report X,x million people (i.e. 4,7 million people)
	
	"""
	if 0 <= x < 100 : return int(x)
	if 100 <= x < 1000 : return round(x / 10) * 10
	if 1000 <= x < 10000 : return round(x / 100) * 100
	if 10000 <= x < 1000000 : return round(x / 1000) * 1000
	if 1000000 <= x : return round(x / 100000) * 100000


Populating the dataframes that will respectively contain the stock disaggregated by broad age groups and sex (`proj_df`), and the rounded figures (`rounded_df`).

In [11]:
proj_df = final_df[final_df['iso3'].isin(countries)]
rounded_df = proj_df.copy()
for index, row in proj_df.iterrows():
	country = row['iso3']
	proj_df.loc[index, age_groups] = proj_df.loc[index, age_groups].multiply(mapping[country] / 100)
	rounded_df.loc[index, age_groups] = proj_df.loc[index, age_groups].apply(rounding)

rounded_df[age_groups] = rounded_df[age_groups].astype(int)

In [12]:
display(proj_df)

,iso3,idmc_short_name,GRID_geographical_group,sex,0-4,0-17,5-14,15-17,15-24,25-64,65+,0+
0,AFG,Afghanistan,South Asia,Female,251941.179931,8.501692e+05,472173.307198,126054.691850,379141.862457,5.733143e+05,50206.000051,1.726777e+06
1,AFG,Afghanistan,South Asia,Male,264879.931306,8.922431e+05,494497.209217,132865.942150,400463.466835,6.166864e+05,43554.388429,1.820081e+06
2,AFG,Afghanistan,South Asia,Total,516821.111237,1.742412e+06,966670.516416,258920.634000,779605.329291,1.190001e+06,93760.388481,3.546858e+06
30,AZE,Azerbaijan,Europe and Central Asia,Total,59849.119649,2.004514e+05,113024.155652,27578.111612,98370.189386,4.146637e+05,49547.816630,7.354550e+05
31,AZE,Azerbaijan,Europe and Central Asia,Female,28219.766719,9.362252e+04,52522.136516,12880.618714,46210.774808,2.123374e+05,28727.577431,3.680177e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
559,UKR,Ukraine,Europe and Central Asia,Male,18268.349651,7.099443e+04,42178.395518,10547.688380,35629.998125,2.026477e+05,41523.555779,3.402480e+05
560,UKR,Ukraine,Europe and Central Asia,Female,17227.322373,6.682346e+04,39743.734949,9852.399694,33648.688145,2.205467e+05,82929.576220,3.940960e+05
591,YEM,Yemen,Middle East and North Africa,Female,245572.931266,8.088752e+05,446053.066220,117249.192118,367689.098575,6.861417e+05,58376.890360,1.803834e+06
592,YEM,Yemen,Middle East and North Africa,Male,255932.087175,8.406253e+05,463846.439899,120846.744157,379023.233864,6.840699e+05,48139.606873,1.831011e+06


###### Description: 2020 Conflict stock by broad age groups and sex.

In [13]:
display(rounded_df)

,iso3,idmc_short_name,GRID_geographical_group,sex,0-4,0-17,5-14,15-17,15-24,25-64,65+,0+
0,AFG,Afghanistan,South Asia,Female,252000,850000,472000,126000,379000,573000,50000,1700000
1,AFG,Afghanistan,South Asia,Male,265000,892000,494000,133000,400000,617000,44000,1800000
2,AFG,Afghanistan,South Asia,Total,517000,1700000,967000,259000,780000,1200000,94000,3500000
30,AZE,Azerbaijan,Europe and Central Asia,Total,60000,200000,113000,28000,98000,415000,50000,735000
31,AZE,Azerbaijan,Europe and Central Asia,Female,28000,94000,53000,13000,46000,212000,29000,368000
...,...,...,...,...,...,...,...,...,...,...,...,...
559,UKR,Ukraine,Europe and Central Asia,Male,18000,71000,42000,11000,36000,203000,42000,340000
560,UKR,Ukraine,Europe and Central Asia,Female,17000,67000,40000,9900,34000,221000,83000,394000
591,YEM,Yemen,Middle East and North Africa,Female,246000,809000,446000,117000,368000,686000,58000,1800000
592,YEM,Yemen,Middle East and North Africa,Male,256000,841000,464000,121000,379000,684000,48000,1800000


###### Description: Rounded 2020 Conflict stock by broad age groups and sex.